# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


import warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,254.63,77,46,3.50,RU,1711514086
1,1,waitangi,-43.9535,-176.5597,289.72,86,98,3.13,NZ,1711514086
2,2,bandar-e lengeh,26.5579,54.8807,293.21,60,40,2.06,IR,1711513894
3,3,sarmiento,-45.5882,-69.0700,286.96,55,98,6.45,AR,1711513894
4,4,dabrowa bialostocka,53.6536,23.3479,276.03,74,61,6.35,PL,1711514048


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                      "Lat",
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      title = "OSM"
                                     )
# Display the map
city_map

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 270)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 1)
                                       & (city_data_df["Wind Speed"] < 4.5)]
# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,iqaluit,63.7506,-68.5145,252.00,77,0,2.57,CA,1711514090
27,27,vadso,70.0744,29.7487,258.24,85,0,2.06,NO,1711514092
212,212,dauphin,51.1494,-100.0502,262.23,57,0,4.12,CA,1711514093
239,239,swift current,50.2834,-107.8014,265.60,67,0,4.12,CA,1711514144
330,330,keflavik,64.0049,-22.5624,269.39,74,0,2.06,IS,1711514163


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,iqaluit,CA,63.7506,-68.5145,77,
27,vadso,NO,70.0744,29.7487,85,
212,dauphin,CA,51.1494,-100.0502,57,
239,swift current,CA,50.2834,-107.8014,67,
330,keflavik,IS,64.0049,-22.5624,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
vadso - nearest hotel: Scandic Vadsø
dauphin - nearest hotel: No hotel found
swift current - nearest hotel: Days Inn
keflavik - nearest hotel: Núpan Deluxe


,City,Country,Lat,Lng,Humidity,Hotel Name
18,iqaluit,CA,63.7506,-68.5145,77,Frobisher Inn
27,vadso,NO,70.0744,29.7487,85,Scandic Vadsø
212,dauphin,CA,51.1494,-100.0502,57,No hotel found
239,swift current,CA,50.2834,-107.8014,67,Days Inn
330,keflavik,IS,64.0049,-22.5624,74,Núpan Deluxe


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)